In [11]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd

import spacy
nlp = spacy.load('en_core_web_sm')

In [12]:
data_amazon = pd.read_csv("amazon_cells_labelled.txt",sep="\t",header = None )
data_imdb = pd.read_csv("imdb_labelled.txt",sep="\t",header = None )
data_yelp = pd.read_csv("yelp_labelled.txt",sep="\t",header = None )

In [13]:
data_yelp.head()


,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [14]:
column_name = ['Review','Sentiment']
data_amazon.columns = column_name
data_amazon.head()


,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [15]:

column_name = ['Review','Sentiment']
data_imdb.columns = column_name
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [16]:
column_name = ['Review','Sentiment']
data_yelp.columns = column_name
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [17]:
# data = data_yelp.append([data_amazon,data_imdb],ignore_index= True)
data = pd.concat([data_amazon, data_imdb])
data = pd.concat([data, data_yelp])
print(data.head())
# print(data)

                                              Review  Sentiment
0  So there is no way for me to plug it in here i...          0
1                        Good case, Excellent value.          1
2                             Great for the jawbone.          1
3  Tied to charger for conversations lasting more...          0
4                                  The mic is great.          1


In [18]:
data.shape


(2748, 2)

In [19]:
data['Sentiment'].value_counts()


Sentiment
1    1386
0    1362
Name: count, dtype: int64

In [20]:
data.isnull().sum()


Review       0
Sentiment    0
dtype: int64

In [21]:
x = data['Review']
y = data["Sentiment"]


In [22]:
import string
punct = string.punctuation
print(punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [23]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)

In [24]:
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = []
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)

  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [25]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")


['hello', 'beautiful', 'day', 'outside']

In [26]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [27]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [28]:
classifier = LinearSVC()

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [30]:
x_train.shape, x_test.shape

((2198,), (550,))

In [31]:
x_train.head()

824                                       Awful service.
526                     horrible, had to switch 3 times.
509    The fat computer geek was unbelievable, the bi...
144    The phone takes FOREVER to charge like 2 to 5 ...
735    Probably never coming back, and wouldn't recom...
Name: Review, dtype: object

In [32]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])


In [33]:
clf.fit(x_train, y_train)

/Users/joonh/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7fec6a8f8ee0>)),
                ('clf', LinearSVC())])

In [34]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [35]:
y_pred = clf.predict(x_test)

In [36]:
confusion_matrix(y_test, y_pred)

array([[221,  60],
       [ 58, 211]])

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79       281
           1       0.78      0.78      0.78       269

    accuracy                           0.79       550
   macro avg       0.79      0.79      0.79       550
weighted avg       0.79      0.79      0.79       550



In [38]:
accuracy_score(y_test, y_pred)

0.7854545454545454

In [39]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])

array([1])

In [40]:
clf.predict(["It's hard to learn new things!"])

array([0])

In [41]:
clf.predict(["Wait, I'm going to die soon!"])

array([0])

In [42]:
clf.predict(["Maht is killing me"])

array([0])